### Import

In [ ]:
import random
import pandas as pd
import time
import json
import yaml
from ultralytics import YOLO
from pathlib import Path

### Yolo model parameters

**EPOCHS**: The number of times the entire training dataset is passed through the model during training. More epochs can help the model learn better, but too many can lead to overfitting.

**IMG_SIZE** (image size): The target size (usually width and height in pixels) to which all training and inference images are resized. Larger sizes can improve detection accuracy but use more memory and computation.

**BATCH** (batch size): The number of images the model processes at once before updating its internal parameters. A larger batch size can speed up training (if you have enough GPU memory) but may require more resources.

In [3]:
DATASET_YAML = Path("../data/silver/")  # point to your dataset.yaml
BASE_MODEL = "yolov8n-cls.pt"
EPOCHS = 50
IMG_SIZE = 512
FRACTION = 1
BATCH = 16
DEVICE = "cpu"
PROJECT_PATH = Path("../runs/classify")

CLASSES = {
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91,
}

### First run of the pre-trained model

In [ ]:
model = YOLO(BASE_MODEL)
random.seed(42)
N = 8 
subset = []
for cls in CLASSES.keys():
    cls_dir = Path(DATASET_YAML) / "val" / cls
    all_imgs = list(cls_dir.glob("*.jpg"))
    if len(all_imgs) >= N:
        subset.extend(random.sample(all_imgs, N))
    else:
        subset.extend(all_imgs)
results = model.predict(source=subset, imgsz=IMG_SIZE, device=DEVICE, save=True, verbose=False, project=PROJECT_PATH, name="food101_base_model",exist_ok=True)
for r in results:
    fname = Path(r.path).name
    pred_class = r.names[r.probs.top1]
    conf = r.probs.top1conf.item()
    print(f"{fname}: {pred_class} ({conf:.2f})")

Results saved to /workspaces/marktguru-home-assignment/runs/base_model
002620_76_pizza.jpg: pizza (0.99)
002667_76_pizza.jpg: pizza (0.86)
002673_76_pizza.jpg: pizza (1.00)
002610_76_pizza.jpg: pizza (0.72)
002675_76_pizza.jpg: pizza (0.96)
002650_76_pizza.jpg: pizza (0.98)
002577_76_pizza.jpg: pizza (0.96)
002504_76_pizza.jpg: pizza (0.99)
025088_90_spaghetti_bolognese.jpg: crayfish (0.91)
025101_90_spaghetti_bolognese.jpg: carbonara (0.38)
025187_90_spaghetti_bolognese.jpg: carbonara (0.80)
025023_90_spaghetti_bolognese.jpg: plate (0.68)
025036_90_spaghetti_bolognese.jpg: plate (0.29)
025201_90_spaghetti_bolognese.jpg: carbonara (1.00)
025011_90_spaghetti_bolognese.jpg: carbonara (1.00)
025068_90_spaghetti_bolognese.jpg: plate (0.50)
022289_91_spaghetti_carbonara.jpg: carbonara (0.95)
022254_91_spaghetti_carbonara.jpg: carbonara (1.00)
022257_91_spaghetti_carbonara.jpg: carbonara (0.99)
022410_91_spaghetti_carbonara.jpg: carbonara (1.00)
022473_91_spaghetti_carbonara.jpg: carbonara (

Pizza and spaghetti carbonara are recognized almost perfectly.
Spaghetti bolognese is missing as a class - confused often with carbonara.

### Fine-tune model with food101 data

The complete set consists of 750 images for training and 250 images for validation in each of three classes.

Eight models will be trained for each constellation of:
- two epoch values: 20 and 50,
- two images sizes: 224 (standard for Yolo) and 512 (images from Hugging Face)
- two dataset lengths: 75 and 750

In [ ]:
# can take hours to run!!!
model = YOLO(BASE_MODEL)
EPOCHS_LIST   = [20, 50]
EPOCHS_LIST   = [50]
IMG_SIZE_LIST = [224,512]
IMG_SIZE_LIST = [512]
FRACTIONS     = [0.1, 1]
for epochs in EPOCHS_LIST:
    for img in IMG_SIZE_LIST:
        for frac in FRACTIONS:
            name = f"food101_e{epochs}_img{img}_frac{frac}"
            print(f"\n▶️ {name}")
            t0 = time.time()
            res = model.train(
                data=str(DATASET_YAML),
                epochs=epochs,
                imgsz=img,
                batch=BATCH,
                device=DEVICE,
                project=str(PROJECT_PATH),
                name=name,
                exist_ok=True,
                fraction=frac,
                verbose=False
            )
            runtime_min = round((time.time() - t0) / 60, 3)
            (PROJECT_PATH / name / "summary.json").write_text(
                json.dumps({"runtime_min": runtime_min}, ensure_ascii=False)
            )
            print(f"⏱️ runtime: {runtime_min} min")

### Gather statistics of model training runs

In [7]:
rows = []
def pick_col(cols, candidates):
    for c in candidates:
        if c in cols: return c
    return None
for run in sorted(PROJECT_PATH.glob("food101_e*_img*_frac*")):
    args_p     = run / "args.yaml"
    results_p  = run / "results.csv"
    summary_p  = run / "summary.json"
    best_pt_p  = run / "weights" / "best.pt"
    if not results_p.exists():
        continue
    args = {}
    if args_p.exists():
        args = yaml.safe_load(args_p.read_text())
    df = pd.read_csv(results_p)
    top1_col = pick_col(df.columns, ["metrics/accuracy_top1", "top1", "val/top1"])
    if top1_col is None:
        best_top1 = float("nan")
    else:
        best_row_idx = df[top1_col].idxmax()
        best_top1 = float(df.loc[best_row_idx, top1_col])
    runtime_min = None
    if summary_p.exists():
        runtime_min = json.loads(summary_p.read_text()).get("runtime_min")
    rows.append({
        "run_dir": str(run.resolve()),
        "epochs": args.get("epochs"),
        "img_size": args.get("imgsz") or args.get("img_size"),
        "fraction": args.get("fraction"),
        "top1_acc_best": round(best_top1, 4) if best_top1 == best_top1 else None,  # NaN guard
        "runtime_min": runtime_min,
        "best_weights": str(best_pt_p.resolve()) if best_pt_p.exists() else None,
    })
stats = pd.DataFrame(rows)
stats_path = PROJECT_PATH / "experiment_stats.csv"
stats.to_csv(stats_path, index=False)

In [ ]:
stats = pd.read_csv(PROJECT_PATH / "experiment_stats.csv")
stats["model"] = stats["best_weights"].apply(lambda p: Path(p).parent.parent.name)
stats_subset = stats.drop(columns=["run_dir", "best_weights"])
(
    stats_subset.style
    .highlight_max(subset=['top1_acc_best'], color='lightgreen')
    .highlight_max(subset=['runtime_min'], color='lightcoral')
    .highlight_min(subset=['top1_acc_best'], color='lightcoral')
    .highlight_min(subset=['runtime_min'], color='lightgreen')
)


,epochs,img_size,fraction,top1_acc_best,runtime_min,model
0,20,224,0.100000,0.333300,6.230000,food101_e20_img224_frac0.1
1,20,224,1.000000,0.972000,28.919000,food101_e20_img224_frac1
2,20,512,0.100000,0.894700,28.893000,food101_e20_img512_frac0.1
3,20,512,1.000000,0.980000,158.982000,food101_e20_img512_frac1
4,50,224,0.100000,0.917300,14.628000,food101_e50_img224_frac0.1
5,50,224,1.000000,0.976000,73.864000,food101_e50_img224_frac1
6,50,512,0.100000,0.940000,71.703000,food101_e50_img512_frac0.1
7,50,512,1.000000,0.970700,420.000000,food101_e50_img512_frac1


### Predict custom uploaded images

In [ ]:
model = YOLO("runs/classify/food101_e20_img224_frac1/weights/best.pt")
model.predict(
    source=Path(DATASET_YAML) / "upload",
    imgsz=224,
    device="cpu",
    save=True,
    project=PROJECT_PATH,
    name="food101_predict_upload"
)